## SRJ

    Update 

        1. Understanding ao2mo and one_ao, two_ao

        2. Basis set isn't issue as I used sto-6g, yet the answer isn't converging

        3. If I am using intor_asymmetric: getting weird results

        4. Tried changing one_ao from intor_symmetric to intor, getting the same energy valus for 5 iterations

        5. For two_ao, initially I used intor("int2e_sph"), going to use ("int2e") -   
            Cartesian Gaussian basis functions - NO CHANGE IN SOLN. 

        6. Core_constant, initially I used rhf.energy_nuc(), when I changed it to mol_pyscf, NO CHANGE IN SOLUTION

        7. Calculated rhf_mo.energy and rhf_mo.occ. I could see that occupancy and energy of MO 23 is low. So going to run the 
            calculation with [17, 18, 19, 20, 21,22] as active. 

        8. Read more about the active space selection. They are optimizing the mo_coeffs and selecting the process. I need them to select them 
            on the basis of rhf.mo_coeff. With a different mo_coeffs,  they are selecting the mo_coeffs based on other MP2 or CCSD calculation 
            and not based on hf 

In [1]:
import basis_set_exchange as bse
print('Check if I am using ANO-RCC-MB basis set')
print('Using newton method ')
basis_dict = bse.get_basis('ANO-RCC-MB', elements=['Fe', 'S'], fmt='nwchem')

import numpy as np
from pyscf import gto, scf, mcscf
from pyscf import gto, ao2mo, scf
import pennylane as qml

#print('Using newton method')
# Your original setup (unchanged) 
mol_pyscf = gto.M(
    atom = 'Fe 0 0 0; S 0 0 1.826',
    basis = basis_dict,
    symmetry = False, 
    spin = 4)
rhf = scf.ROHF(mol_pyscf)
print('Molecular point group:', mol_pyscf.topgroup)
rhf.max_cycle = 5000
#rhf.init_guess = "hcore"
#rhf.level_shift = 0.0    #This command is changing the energy 
#rhf.damp = 0.2
#rhf.diis_start_cycle = 5

#rhf = rhf.newton() 
e = rhf.kernel()



for i, (e, occ) in enumerate(zip(rhf.mo_energy, rhf.mo_occ)):
    print(f"MO #{i:2d}: Energy = {e:10.6f} Ha, Occupancy = {occ}")

import pennylane as qml 
from pennylane import qchem

#Atomic integrals - calculations done and understood 
one_ao = mol_pyscf.intor_symmetric("int1e_kin") + mol_pyscf.intor_symmetric("int1e_nuc")   #intor_symmetric : One -electron integral generator. Integrals are assumed here to be hermitian (https://pyscf.org/_modules/pyscf/gto/mole.html#MoleBase.intor_symmetric)
two_ao = mol_pyscf.intor("int2e_sph")  #eri electron repulsion integrals - Tried using int2e, still same results

#Molecular integrals 
one_mo = np.einsum("pi,pq,qj->ij", rhf.mo_coeff, one_ao, rhf.mo_coeff, optimize=True)
two_mo = ao2mo.incore.full(two_ao, rhf.mo_coeff)




core_constant = np.array([rhf.energy_nuc()])


# convert the two-electron integral tensor to the physicists’ notation
two_mo = np.swapaxes(two_mo, 1, 3)
#print('Shape after swapping axes', two_mo.shape)

#Core and active are received from the previous code
core = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,17]
active = [ 18,19, 20, 21, 22, 23]

total_orb = len(core) + len(active)
print('Total orbitals are',total_orb)



# define the active space and recompute the integrals
def pyscf_integral(core, active, one_mo, two_mo, core_constant):
    if core and active:
        for i in core:
            #print('The value of i going in ', i)
            core_constant = core_constant + 2 * one_mo[i][i]
            #print('core cosntant', core_constant)
            for j in core:
                #print('Value of i and j ', i,j)
                core_constant = core_constant + 2 * two_mo[i][j][j][i] - two_mo[i][j][i][j]
        print('The final value of core constant', core_constant)

        for p in active:
            for q in active:
                for i in core:
                    #print(f'The value of p and q', {p}, {q})
                    one_mo[p, q] = one_mo[p, q] + (2 * two_mo[i][p][q][i] - two_mo[i][p][i][q])

        one_mo = one_mo[qml.math.ix_(active, active)]
        two_mo = two_mo[qml.math.ix_(active, active, active, active)]
        print('Within pyscf integral one_mo shape', one_mo.shape)
        print('Within pyscf integral one_mo shape', one_mo.shape)
    return core_constant, one_mo, two_mo

core_constant, one_mo, two_mo = pyscf_integral(core, active, one_mo,two_mo, core_constant)


H_fermionic = qml.qchem.fermionic_observable(core_constant, one_mo, two_mo, cutoff=1e-20)

H = qml.jordan_wigner(H_fermionic)

print(H)

Check if I am using ANO-RCC-MB basis set
Using newton method 
Molecular point group: C1
converged SCF energy = -1655.34968383623
MO # 0: Energy = -259.180107 Ha, Occupancy = 2.0
MO # 1: Energy = -91.641815 Ha, Occupancy = 2.0
MO # 2: Energy = -31.779132 Ha, Occupancy = 2.0
MO # 3: Energy = -27.441840 Ha, Occupancy = 2.0
MO # 4: Energy = -27.441840 Ha, Occupancy = 2.0
MO # 5: Energy = -27.420625 Ha, Occupancy = 2.0
MO # 6: Energy =  -8.859585 Ha, Occupancy = 2.0
MO # 7: Energy =  -6.547802 Ha, Occupancy = 2.0
MO # 8: Energy =  -6.541798 Ha, Occupancy = 2.0
MO # 9: Energy =  -6.541798 Ha, Occupancy = 2.0
MO #10: Energy =  -4.269029 Ha, Occupancy = 2.0
MO #11: Energy =  -2.875990 Ha, Occupancy = 2.0
MO #12: Energy =  -2.866330 Ha, Occupancy = 2.0
MO #13: Energy =  -2.866330 Ha, Occupancy = 2.0
MO #14: Energy =  -0.897661 Ha, Occupancy = 2.0
MO #15: Energy =  -0.646508 Ha, Occupancy = 2.0
MO #16: Energy =  -0.390507 Ha, Occupancy = 2.0
MO #17: Energy =  -0.390507 Ha, Occupancy = 2.0
MO #18

In [2]:
perfect_core = -1644.343471612231
core_constant - perfect_core

array([6.82121026e-13])

    1. Correct frozen core = -1644.343471612231

In [2]:
m = gto.M(atom='Fe 0 0 0; S 0 0 1.826', 
    basis=basis_dict, spin=4, charge=0)
hf = m.ROHF()
hf.kernel()
for j in range(10):
    mo = hf.stability()[0]
    if np.allclose(mo, hf.mo_coeff):
       break
    dm = hf.make_rdm1(mo, hf.mo_occ)
    hf = hf.run(dm)

h1e = m.intor("int1e_kin") + m.intor("int1e_nuc")
h2e = m.intor("int2e")
scf_c = hf.mo_coeff
print('scf_c', scf_c)
nuclear_repulsion = m.energy_nuc()
constant = nuclear_repulsion

converged SCF energy = -1655.34968383623
<class 'pyscf.scf.rohf.ROHF'> wavefunction is stable in the internal stability analysis
scf_c [[ 9.99956465e-01  3.24327320e-06 -8.67865689e-03 -2.90072984e-13
   3.80043782e-13  8.85039647e-07 -2.36660166e-05  2.36538713e-05
   8.88286957e-14 -1.09633030e-13  3.03998007e-03 -1.16765284e-04
  -3.10919893e-12  2.62155659e-12 -2.54691412e-03  2.22503471e-04
  -2.18768351e-10  2.52421745e-10  3.80300131e-03 -8.35400311e-16
  -2.65257840e-09 -1.79081863e-10  2.03839207e-10  2.44540471e-03
  -2.25054140e-12  3.13493146e-13  5.92744286e-03]
 [ 8.71458149e-03  2.38912596e-05  9.99895404e-01  6.30956254e-11
  -8.55989821e-11  7.41492416e-04 -1.76195440e-04  1.92784126e-04
   7.45558918e-13 -9.38676230e-13 -1.29357516e-02 -6.23479524e-04
  -1.24083682e-11  8.55351068e-12 -1.57621534e-02  8.52353586e-04
  -1.23944669e-09  1.42966278e-09  2.15939226e-02 -5.98534157e-15
  -1.51383180e-08 -1.03577550e-09  1.17907070e-09  2.09188829e-02
  -1.43535789e-11  1.8

In [ ]:
mol = gto.M(
    atom='Fe 0 0 0; S 0 0 1.826',
    basis='sto-3g',
    spin=4,
    symmetry=True  # Turn on symmetry detection
)
print('Molecular point group:', mol.topgroup)

#(-1654.1650027396465+0j) * I(0) + (0.1512819994239989+0j) * Z(0) + (1.1542687183696362e-08+0j)(Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) - intor_symmetric
#intor: 
#(-1654.1650027396465+0j) * I(0) + (0.1512819994239989+0j) * Z(0) + -1.1272420341178636e-16j * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ X(4))

Molecular point group: Coov


In [3]:
#Check the possible excitations
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import itertools
import time
import re
import scipy
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []
old_grad = []
excitationlist = []
generatingfns = []
gs_energy = []
operator_check = []  # To store the highest gradient excitation operators
theta = np.pi/4
print('Theta is', theta)

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity




#symbols = ["H","H","H","H"]
electrons = 6
qubits = 12
#orbitals = 8
#r_bohr = bond_length *1.8897259886 
##r_bohr = bond_length * 1.0  # Convert bond length to Bohr radius 
#coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
#H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-3g", method="pyscf")
#hf_state = qchem.hf_state(electrons, qubits)

def adaptvqe(adapt_it = 4, e_th=1e-12):
    ash_excitation = []
    energies = []
    excitations= []
    #Preparation of the Hartree-Fock state in the form of 2^8 states
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def hf_stateprep(wires):
        target_state = np.zeros(2**qubits)
        target_state[4032] = 1.0 # Every molecule change, you need to change this index
        qml.StatePrep(target_state, wires=range(qubits))
        return qml.state()

    hf_state = hf_stateprep(wires=qubits)
    #print('HF state is', hf_state)

    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, electrons, qubits, H):
        #print('Updated hf_state is', hf_state)  
        qml.StatePrep(hf_state, wires=range(qubits))
        return qml.expval(H)  
    
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)
    #Applyign the givens rotation to the HF state
    @qml.qnode(dev)
    def new_state(hf_state, ash_excitation, qubits):
        qml.StatePrep(hf_state, wires=range(qubits))
        for i in range(len(ash_excitation)):
            if len(ash_excitation[i]) == 4:
                qml.DoubleExcitation(theta, wires=ash_excitation[i])
            elif len(ash_excitation[i]) == 2:
                qml.SingleExcitation(theta, wires=ash_excitation[i])
        return qml.state()
    
    #Measurement of new state
    dev_meas = qml.device("lightning.qubit", wires=qubits, shots=1000)
    @qml.qnode(dev_meas)
    def measure(ostate):
        qml.StatePrep(ostate, wires=range(qubits))
        return qml.counts()
    
    @qml.qnode(dev)
    def ind_state(ash_excitation):
        qml.StatePrep(hf_state, wires=range(qubits))
        print('Individual excitation is', ash_excitation)
        if len(ash_excitation) == 4 :
            qml.DoubleExcitation(theta, wires=ash_excitation)
        elif len(ash_excitation) == 2:
            qml.SingleExcitation(theta, wires=ash_excitation)
        return qml.state()


    print('HF state is', circuit(hf_state, electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(electrons, qubits)

    
    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('The operator pool length is', len(operator_pool))
    states = [hf_state]
    max_operator = None

    for j in range(1, adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        #max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
        counts1 = measure(k)  #Measure the current state
        print('The measurement of new state is', counts1)
        print(f'The operator check for this iteration {j} is', operator_check)
        for i in operator_pool:
            if i not in operator_check:  # Check if the operator is already used
                #print('The current excitation operator is', i) #Current excitation operator - fermionic one
                #print('The value of  operator check is', operator_check)  #Max excitation operator
                w = qml.fermi.jordan_wigner(i)  #JW transformation
                if np.array_equal(k, hf_state): # If the current state is the HF state
                    current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
                else:
                    current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
                #print(f'The expectation value of {i} is', current_value)

                if current_value > max_value:
                    max_value = current_value
                    max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value
        operator_check.append(max_operator)
        # Convert operator to excitations and append to ash_excitation
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations1 = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations1)
        ash_excitation.append(excitations1)
        print('ash_excitation is', ash_excitation)
        print('The length of ash_excitation before generating matrix is', len(ash_excitation))

        
        #Now apply the givens rotation for it. 
        ostate = new_state(hf_state, ash_excitation, qubits)
        #print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation, qubits))
        #Measuring the state
        counts = measure(ostate)
        print('State count after Generating function is', counts)
        if j >= 2:
            states.append(ind_state(ash_excitation[-1]))  # Append the individual excitation state
        else:
            print('Skipping individual excitation state for j < 2')
        states.append(ostate)
        print(f'After adding the states the adapt iteration is {j} and no of states',len(states))
        M = np.zeros((len(states), len(states)), dtype=complex)  # Initialize H with zeros
        S = np.zeros((len(states), len(states)), dtype=complex)  # Initialize S matrix with zeros
        Ham_matrix = qml.matrix(H, wire_order=range(qubits))  # Get the Hamiltonian matrix
        #print('Shape of Hamiltonian matrix is', Ham_matrix.shape)
        # Now we need to fill the H and S matrices
        for i in range(len(states)):
            for j in range(len(states)):
                #print('The value of i and j is', i, j)
                left_op = states[i].T.conj()
                right_op = states[j]
                M[i,j] = left_op.dot(Ham_matrix.dot(right_op)).real
                S[i,j] = left_op.dot(right_op).real
     
                
                if i != j:
                    M[j,i] = M[i,j].conj()
                    S[j,i] = S[i,j].conj()
        n = S.shape[0]
        epsilon = 1e-10
        S_reg = S + epsilon * np.eye(n)
        #print(f'S matrix with {i} and {j} is', S)
        #print('Hamiltonian Matrix with', i, j, 'is', M)
        #print('The shape of S matrix is', S.shape)  
        eig,evec=scipy.linalg.eigh(M,S_reg)
        #print('Eigenvalues are', eig)
        gs_energy.append(eig[0])
        print('Ground state energy is', gs_energy) 
        print('\n')
          
    return ash_excitation, states,eig,gs_energy, Ham_matrix,hf_state


Theta is 0.7853981633974483


In [4]:
ash_excitation, states,eig,gs_energy,Ham_matrix, hf_state = adaptvqe(adapt_it=100, e_th=1e-12)

/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


HF state is -1655.16339069357
The operator pool length is 117
The adapt iteration now is 1
The measurement of new state is {np.str_('111111000000'): np.int64(1000)}
The operator check for this iteration 1 is []
The highest operator value is 0.10812497583517337 for operator a⁺(0) a⁺(1) a(10) a(11)
Highest gradient excitation is [0, 1, 10, 11]
ash_excitation is [[0, 1, 10, 11]]
The length of ash_excitation before generating matrix is 1
State count after Generating function is {np.str_('001111000011'): np.int64(140), np.str_('111111000000'): np.int64(860)}
Skipping individual excitation state for j < 2
After adding the states the adapt iteration is 1 and no of states 2
Ground state energy is [np.float64(-1655.1697677330428)]


The adapt iteration now is 2
The measurement of new state is {np.str_('001111000011'): np.int64(128), np.str_('111111000000'): np.int64(872)}
The operator check for this iteration 2 is [FermiWord({(0, 0): '+', (1, 1): '+', (2, 10): '-', (3, 11): '-'})]
The highest o

In [5]:
gs_energy[-1]

np.float64(-1655.3537936470298)

In [6]:
FCI = -1655.3586136786419

error = gs_energy[-1] - FCI
print('Final error', error)

Final error 0.004820031612098319
